## Constants

In [1]:
import sys, os
import pandas as pd
import polars as pl
import numpy as np
import subprocess
import gc
import optuna
from datetime import datetime, timezone
import warnings
import xgboost as xgb
import joblib as jl
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import matthews_corrcoef
from mlflow.models import infer_signature
import mlflow
import random
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import StratifiedKFold

today = datetime.now(timezone.utc).strftime("%Y_%m_%d")
warnings.filterwarnings("ignore")

from hyper_params import (
    mushroom_tuning_2024_08_06_1722934727_params,
)

# helpers


SEED = 108
random.seed(SEED)
N_FOLDS = 7
# data
train_path = "../../data/mushrooms/train.csv"
test_path = "../../data/mushrooms/test.csv"
cache_path = "../../data/mushrooms/cache"
# model
is_tunning = True
try:
    rs = subprocess.check_output("nvidia-smi")
    device = "cuda" if rs is not None else "cpu"
except (
    Exception
):  # this command not being found can raise quite a few different errors depending on the configuration
    print("No Nvidia GPU in system!")
    device = "cpu"
goal = "binary:logistic"

# custom metric
objective_dict = {
    "binary:logistic": {
        "metric": {
            "is_custom": False,
            "name": "logloss",
            "fval": None,
        },
        "direction": "minimize",
    }
}
# objective_dict = {
#     "binary:logistic": {
#         "metric": {
#             "is_custom": True,
#             "name": "MCC",
#             "fval": mcc_metric_v2,
#         },
#         "direction": "maximize",
#     }
# }
metric = objective_dict[goal]["metric"]["name"]
is_custom_metric = objective_dict[goal]["metric"]["is_custom"]
fval = objective_dict[goal]["metric"]["fval"]
direction = objective_dict[goal]["direction"]

best_params = {
    "device": device,
    "verbosity": 0,
    "objective": goal,
}
best_params.update(mushroom_tuning_2024_08_06_1722934727_params)
best_params

{'device': 'cuda',
 'verbosity': 0,
 'objective': 'binary:logistic',
 'tree_method': 'hist',
 'eta': 0.0696294726051571,
 'max_depth': 0,
 'min_child_weight': 1,
 'gamma': 0.044230646284796976,
 'subsample': 0.9405269471473167,
 'colsample_bytree': 0.2999355523666192,
 'lambda': 0.9746051811186938,
 'alpha': 4.210861941737071}

## Prepare data

In [2]:
train = pd.read_csv(train_path)
print(f"train size: {train.shape}")
X_test = pd.read_csv(test_path)
print(f"test size: {X_test.shape}")
submit_df = pd.DataFrame()
submit_df["id"] = X_test["id"]
y_train = train["class"]
X_train = train.drop(columns=["id", "class"], axis=1)
X_test.drop(columns=["id"], inplace=True, axis=1)

train size: (3116945, 22)
test size: (2077964, 21)


In [3]:
target = "class"

categorical_cols = (
    train.drop(columns=target).select_dtypes(include="object").columns.to_list()
)
# for c in categorical_cols:
#     train[c] = train[c].astype("category")
#     X_test[c] = X_test[c].astype("category")
numerical_cols = (
    train.drop(columns="id").select_dtypes(include="number").columns.to_list()
)
gc.collect()

201

## Data preprocessing

In [4]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder

# get top 10 most frequent names
n = 10
for c in categorical_cols:
    train_mode_values = X_train[c].value_counts()[:n].index.tolist()
    X_train.loc[~X_train[c].isin(train_mode_values), c] = "other"
    test_mode_values = X_test[c].value_counts()[:n].index.tolist()
    X_test.loc[~X_test[c].isin(test_mode_values), c] = "other"


# Create the numerical and categorical pipelines
numerical_pipeline = Pipeline(
    steps=[
        ("num_imputer", SimpleImputer(strategy="median")),
        # ("minmax", MinMaxScaler()),
        ("scaler", StandardScaler()),
    ]
)

categorical_pipeline = Pipeline(
    steps=[
        ("cat_imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
        # ("ordinal", OrdinalEncoder()),
    ]
)

# Combine the pipelines into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_pipeline, numerical_cols),
        ("cat", categorical_pipeline, categorical_cols),
    ]
)

# Create the full pipeline with the XGBoost model
data_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
    ]
)
data_pipeline.set_output(transform="polars")
# Preprocess the data
X_all = pd.concat([X_train, X_test])
data_pipeline.fit(pl.from_pandas(X_all))
X_test_transformed = data_pipeline.transform(X_test)
X_train_transformed = data_pipeline.transform(X_train)
# Binarize the target labels
lb = LabelBinarizer()

y_train_binarized = lb.fit_transform(y_train)

print(f"Exporting to pickple...")
jl.dump(X_train_transformed, "X_train.pkl")
jl.dump(y_train_binarized, "y_train.pkl")
jl.dump(X_test_transformed, "X_test.pkl")
jl.dump(submit_df, "submit_df.pkl")
jl.dump(lb, "lb.pkl")

gc.collect()

gc.collect()

Exporting to pickple...


0

## CV

In [5]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


clf: xgb.XGBClassifier = xgb.XGBClassifier(
    **best_params,
    n_estimators=4000,
    early_stopping_rounds=50,
    enable_categorical=True,
)

In [6]:
from tqdm import tqdm

skf = StratifiedKFold(n_splits=N_FOLDS)

# kf = KFold(n_splits=N_FOLDS)
y_preds = []
y_trues = []
for train_index, test_index in tqdm(skf.split(X_train_transformed, y_train_binarized)):
    X_train, X_test = X_train_transformed[train_index], X_train_transformed[test_index]
    y_train, y_test = y_train_binarized[train_index], y_train_binarized[test_index]

    clf.fit(X=X_train, y=y_train, eval_set=[(X_test, y_test)])

    y_pred = clf.predict(X_test)
    y_preds.append(y_pred)
    y_trues.append(y_test)
# Concatenate the predictions and true labels
y_preds_concat = np.concatenate(y_preds)
y_trues_concat = np.concatenate(y_trues)
mcc = matthews_corrcoef(y_trues_concat, y_preds_concat)
print(f"Validation mcc score: {mcc}")

0it [00:00, ?it/s]

[0]	validation_0-logloss:0.66598
[1]	validation_0-logloss:0.63139
[2]	validation_0-logloss:0.59327
[3]	validation_0-logloss:0.56558
[4]	validation_0-logloss:0.53150
[5]	validation_0-logloss:0.49668
[6]	validation_0-logloss:0.48240
[7]	validation_0-logloss:0.45914
[8]	validation_0-logloss:0.44656
[9]	validation_0-logloss:0.43270
[10]	validation_0-logloss:0.41277
[11]	validation_0-logloss:0.38505
[12]	validation_0-logloss:0.36878
[13]	validation_0-logloss:0.36104
[14]	validation_0-logloss:0.34888
[15]	validation_0-logloss:0.33579
[16]	validation_0-logloss:0.32745
[17]	validation_0-logloss:0.32105
[18]	validation_0-logloss:0.30682
[19]	validation_0-logloss:0.29582
[20]	validation_0-logloss:0.28440
[21]	validation_0-logloss:0.27317
[22]	validation_0-logloss:0.26828
[23]	validation_0-logloss:0.25714
[24]	validation_0-logloss:0.24486
[25]	validation_0-logloss:0.23018
[26]	validation_0-logloss:0.22298
[27]	validation_0-logloss:0.21830
[28]	validation_0-logloss:0.20756
[29]	validation_0-loglos

1it [00:20, 20.46s/it]

[0]	validation_0-logloss:0.66598
[1]	validation_0-logloss:0.63141
[2]	validation_0-logloss:0.59324
[3]	validation_0-logloss:0.56545
[4]	validation_0-logloss:0.53131
[5]	validation_0-logloss:0.49653
[6]	validation_0-logloss:0.48224
[7]	validation_0-logloss:0.45895
[8]	validation_0-logloss:0.44636
[9]	validation_0-logloss:0.43246
[10]	validation_0-logloss:0.41252
[11]	validation_0-logloss:0.38479
[12]	validation_0-logloss:0.36855
[13]	validation_0-logloss:0.36080
[14]	validation_0-logloss:0.34869
[15]	validation_0-logloss:0.33562
[16]	validation_0-logloss:0.32727
[17]	validation_0-logloss:0.32092
[18]	validation_0-logloss:0.30669
[19]	validation_0-logloss:0.29578
[20]	validation_0-logloss:0.28434
[21]	validation_0-logloss:0.27311
[22]	validation_0-logloss:0.26820
[23]	validation_0-logloss:0.25709
[24]	validation_0-logloss:0.24484
[25]	validation_0-logloss:0.23015
[26]	validation_0-logloss:0.22295
[27]	validation_0-logloss:0.21825
[28]	validation_0-logloss:0.20751
[29]	validation_0-loglos

2it [00:40, 20.07s/it]

...

[0]	validation_0-logloss:0.66605
[1]	validation_0-logloss:0.63148
[2]	validation_0-logloss:0.59327
[3]	validation_0-logloss:0.56553
[4]	validation_0-logloss:0.53138
[5]	validation_0-logloss:0.49655
[6]	validation_0-logloss:0.48226
[7]	validation_0-logloss:0.45891
[8]	validation_0-logloss:0.44634
[9]	validation_0-logloss:0.43253
[10]	validation_0-logloss:0.41259
[11]	validation_0-logloss:0.38489
[12]	validation_0-logloss:0.36864
[13]	validation_0-logloss:0.36087
[14]	validation_0-logloss:0.34871
[15]	validation_0-logloss:0.33562
[16]	validation_0-logloss:0.32894
[17]	validation_0-logloss:0.32251
[18]	validation_0-logloss:0.30817
[19]	validation_0-logloss:0.29712
[20]	validation_0-logloss:0.28559
[21]	validation_0-logloss:0.27427
[22]	validation_0-logloss:0.26933
[23]	validation_0-logloss:0.25816
[24]	validation_0-logloss:0.24582
[25]	validation_0-logloss:0.23102
[26]	validation_0-logloss:0.22378
[27]	validation_0-logloss:0.21907
[28]	validation_0-logloss:0.20826
[29]	validation_0-loglos

6it [01:59, 20.05s/it]

[0]	validation_0-logloss:0.66590
[1]	validation_0-logloss:0.63126
[2]	validation_0-logloss:0.59311
[3]	validation_0-logloss:0.56538
[4]	validation_0-logloss:0.53128
[5]	validation_0-logloss:0.49646
[6]	validation_0-logloss:0.48220
[7]	validation_0-logloss:0.45891
[8]	validation_0-logloss:0.44630
[9]	validation_0-logloss:0.43244
[10]	validation_0-logloss:0.41258
[11]	validation_0-logloss:0.38483
[12]	validation_0-logloss:0.36859
[13]	validation_0-logloss:0.36082
[14]	validation_0-logloss:0.34870
[15]	validation_0-logloss:0.33562
[16]	validation_0-logloss:0.32729
[17]	validation_0-logloss:0.32089
[18]	validation_0-logloss:0.30665
[19]	validation_0-logloss:0.29565
[20]	validation_0-logloss:0.28418
[21]	validation_0-logloss:0.27296
[22]	validation_0-logloss:0.26804
[23]	validation_0-logloss:0.25693
[24]	validation_0-logloss:0.24467
[25]	validation_0-logloss:0.22998
[26]	validation_0-logloss:0.22277
[27]	validation_0-logloss:0.21809
[28]	validation_0-logloss:0.20737
[29]	validation_0-loglos

7it [02:20, 20.07s/it]


Validation mcc score: 0.9846544045237705


In [7]:
y_preds = clf.predict(X_test_transformed)
pred_classes = lb.inverse_transform(y_preds)
submit_df["class"] = pred_classes
submit_df.to_csv("submission.csv", index=False)